In [6]:
import os #operating system module for file IO
import csv #csv module for spreadsheets

import itertools #tools forr iterating data

import pandas as pd #database module
import hashlib #consistent hashing for persistence

import numpy as np
import scipy
import time
import plotly.graph_objects as go

import pylab
import matplotlib
from matplotlib.pyplot import figure
from mpl_toolkits import mplot3d
import numpy as np

from qiskit import Aer
from qiskit.opflow import X, Z, Y, I
from qiskit.opflow.primitive_ops import PauliOp, PauliSumOp
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.circuit.library import TwoLocal, QAOAAnsatz

from qiskit.algorithms.optimizers import ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, NELDER_MEAD, \
                                            NFT, POWELL, SLSQP, SPSA, TNC

np.set_printoptions(formatter={'float': '{:0.3f}'.format})

In [ ]:
#Sets up optimizers
all_optimizers = [ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, NELDER_MEAD, NFT, POWELL, SLSQP, SPSA, TNC]

def setOptimizers(optimizers, max_iters):
    all_optimizers = [ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, \
                  NELDER_MEAD, NFT, POWELL, SLSQP, SPSA, TNC]
    
    bool_optimizers = [o in optimizers for o in all_optimizers]
    
    for i, optimizer in enumerate(all_optimizers):
        if bool_optimizers[i]:
            print(f"\x1b[32m{optimizer.__name__:15} ON")
        else: 
            print(f"\x1b[31m{optimizer.__name__:15} OFF")
            
    optimizers = [o(maxiter=max_iters) for o in optimizers]
        
    return optimizers

In [6]:
#Builds a Pauli term by defining which pauli you want at what ndx and the size of the term
def buildMonomial(pauli, ndx, size):
    result = pauli if ndx == 0 else I
    for i in range(1, size):
        result ^= pauli if i == ndx else I
    return result

def buildTerm(size, pauli, coeff = 1.0):
    result = pauli.get(0) or I
    for i in range(1, size):
        result ^= pauli.get(i) or I
    return result * coeff
    
#converts Coupling Matrix -> PauliSumOp
def buildHamiltonian(coupling_mat):
    result = None
    paulis = [X, Y, Z]
    size = len(coupling_mat) // 3
    
    dim = len(coupling_mat.shape)
    num_entries = np.prod(coupling_mat.shape)
    
    for i in range(num_entries):
        f = lambda i,j : i // ((size*3)**j)
        loc = tuple([(f(i,j) % (size*3)) for j in range(dim)])
        
        if coupling_mat[loc] != 0 and tuple(np.sort(loc)) == loc:
            pauli_map = [paulis[aisle // size] for aisle in loc]
            ndx_map = [aisle % size for aisle in loc]
            ndx_pauli = dict(zip(ndx_map, pauli_map))
            term = buildTerm(size, ndx_pauli, coupling_mat[loc])
            result = result + term if result else term
    return result

In [7]:
# Wrapper for Coupling Matrix to make some things easier
# Values are not split across diagonal
class CouplingMatrix:
    
    def __init__(self, N, I_coeff = 0):
        self.qubits = N
        self.size = 3 * N
        self.matrix = np.zeros((self.size, self.size), dtype=float)
        self.I_coeff = I_coeff
        self.set_submatrices()
        self.constant = 0
    
    def __setitem__(self, key, value):
        self.matrix[key] = value
        self.matrix[key[::-1]] = value
        self.set_submatrices()
        
    def __getitem__(self, key):
        return self.matrix[key]
    
    def __str__(self):
        return self.matrix.__str__()
    
    def __repr__(self):
        return self.matrix.__str__()
    
    def getHamiltonian(self):
        hamiltonian = buildHamiltonian(self.matrix)
        if hamiltonian:
            if self.I_coeff:
                return hamiltonian + buildMonomial(I, 0, self.qubits) * self.I_coeff
            else:
                return hamiltonian
    
    def set_submatrices(self):
        self.X = self.matrix[:self.qubits,:self.qubits]
        self.Y = self.matrix[self.qubits:2*self.qubits,self.qubits:2*self.qubits]
        self.Z = self.matrix[2*self.qubits:,2*self.qubits:]

In [ ]:
#Builds Schwinger Model Coupling Matrices
def make_schwinger(N, H_E_coeff, H_M_coeff, H_I_coeff, theta = 0, mixer_type = 'XY'):
    assert N >= 2, "N has to be greater than or equal to 2"
    assert N <= 22, "N can be no greater than 22"
    assert N % 2 == 0, "N must be even"
    assert mixer_type in {'X', 'Y', 'XY'}, "mixer_type must be 'X', 'Y', or 'XY'"
    
    #Constant
    I_coeff = (1/4) * ((N-1) * (theta/np.pi)**2 + np.ceil((N-1)/2.0) * (1 + 2*(theta/np.pi)) + ((N*(N-1))/2)) * H_E_coeff

    mixer_coupling = CouplingMatrix(N)    
    target_coupling = CouplingMatrix(N, I_coeff)

    #H_E
    for j in range(N-1):
        for k in range(j):
            target_coupling[j + 2*N, k + 2*N] += (N - j - 1)/2.0 * H_E_coeff

    for j in range(N):
        target_coupling[j + 2*N,j + 2*N] += (1/2.0) * ((theta/np.pi) * (N-j-1) + (np.ceil((N - j - 1)/2) - ((N*j)%2) )) * H_E_coeff

    #H_M
    for i in range(N):
        target_coupling[i + 2*N,i + 2*N] += ((-1) ** i) * H_M_coeff

    #H_I
    for i in range(N-1):
        if mixer_type in {'XY', 'X'}:
            mixer_coupling[i,i + 1] += H_I_coeff
        if mixer_type in {'XY', 'Y'}:  
            mixer_coupling[i + N,i + 1 + N] += H_I_coeff
            
    target_ham = target_coupling.getHamiltonian()
    mixer_ham = mixer_coupling.getHamiltonian()
        
    return target_ham, mixer_ham

# Rendering Landscape

In [ ]:
def find_min_coeff(ham):
    if type(ham) == PauliOp:
        min_coeff = abs(ham.coeff)
    else:
        min_coeff = min(abs(ham.coeffs))
    return min_coeff

# File IO

# Find Dimensionless Energy

Given the first three eigenvalues of the Schwinger Model
$$\omega_0, \omega_\_, \omega_+$$

We find the dimensionless Eigenvalues

$$\begin{align}
    f_0 &= \frac{\omega_0}{2Nx} \\
    f_\_ &=  \frac{\omega_\_ - \omega_0}{2\sqrt(x)} - 2\frac{m}{g} \\
    f_+ &= \frac{\omega_+ - \omega_0}{2\sqrt(x)} - 2\frac{m}{g}
\end{align}$$

In [1]:
#Calculates the lowest two eigenvalues, w0 and w1
def findDimensionlessEigen(x, N, m):
    N = int(N)
    m = m
    target_coupling, mixer_coupling = make_schwinger(N, 1, np.sqrt(x) / 2 * m, x/2.0)

    mixer_hamiltonian = mixer_coupling.getHamiltonian()
    target_hamiltonian = target_coupling.getHamiltonian()

    complete_hamiltonian = mixer_hamiltonian + target_hamiltonian

    sparse_matrix_ham = complete_hamiltonian.to_spmatrix()
    eigenvalues = np.sort(scipy.sparse.linalg.eigsh(sparse_matrix_ham, return_eigenvectors=False, which = 'SA', k = 3))
    
    w0, w1, w2 = eigenvalues[:3]
    f0, f1, f2 = w0/(2*N*x), ((w1-w0)/(2*np.sqrt(x))) - 2*m, ((w2-w0)/(2*np.sqrt(x))) - 2*m
    
    return f0, f1, f2

In [ ]:
def dimensionless_eigv(N, x, m):
    N = int(N)
    target_coupling, mixer_coupling = make_schwinger(N, 1, np.sqrt(x) / 2 * m, x/2.0)

    mixer_hamiltonian = mixer_coupling.getHamiltonian()
    target_hamiltonian = target_coupling.getHamiltonian()

    complete_hamiltonian = mixer_hamiltonian + target_hamiltonian

    matrix_ham = complete_hamiltonian.to_matrix(massive = True)
    
    return np.sort(np.linalg.eigvalsh(matrix_ham))

# Extrapolation

In [1]:
def gen_extrapolated_data(x_data, y_data, z_data, extrap_range, extrap_axis, poly_size = 2, get_intercept = False):
    
    assert extrap_axis == 'y' or extrap_axis =='x'
    
    x_data, y_data = x_data[0], y_data.T[0]
    
    if extrap_axis == 'y':
        x_range, y_range = len(x_data), len(extrap_range)
        x_data_extrap, y_data_extrap = np.meshgrid(x_data, extrap_range)
        dep_axis = np.array(y_data)
    else:
        x_range, y_range = len(y_data), len(extrap_range)
        x_data_extrap, y_data_extrap = np.meshgrid(extrap_range, y_data)
        dep_axis = np.array(x_data)

    poly_data = [np.zeros([x_range, y_range], dtype=float) for _ in range(poly_size)]
    
    if extrap_axis == 'y':
        poly_intercepts = np.zeros([poly_size, len(z_data)])
    
    for i in range(len(z_data)):
        indep_axis = np.array(z_data[i])
        intercept = []
        for p in range(poly_size):
            coeffs = np.polyfit(dep_axis, indep_axis, p + 1)
            poly_intercepts[p, i] = coeffs[-1]
            poly_data[p][i] = np.poly1d(coeffs)(extrap_range)
            
    if extrap_axis == 'y':
        poly_data = [poly.T for poly in poly_data]
    
    if get_intercept:
        return x_data_extrap, y_data_extrap, poly_data, poly_intercepts
    return x_data_extrap, y_data_extrap, poly_data

In [21]:
def check_iterable(inp):
    '''
    Checks whether an input is iterable
    '''
    try:
        iterator = iter(inp)
    except TypeError:
        return False
    else:
        return type(inp) != str

def check_float(num):
    '''
    Checks whether a string can be interpreted as an integer or float
    '''
    assert type(num) == str, "Only checks strings"
    try:
        fnum = float(num)
    except ValueError:
        return False
    else:
        return True

In [3]:
def hash_iterable(t):
    '''
    hashes an iterable
    '''
    m = hashlib.md5()
    for s in t:
        m.update(str(s).encode())
    fn = m.hexdigest()
    return fn

In [ ]:
def set_range(xrr = (0,1), yrr = (0,1)):
    size = renders[list(renders)[0]][0].shape
    xrr = (int(np.ceil(xrr[0] * size[0])),int(np.ceil(xrr[1] * size[0])))
    yrr = (int(np.ceil(yrr[0] * size[1])), int(np.ceil(yrr[1] * size[1])))
    return xrr, yrr

def dict_cartesian_product(dic):
    '''
    Makes a cartesian product of a dictionary that has lists for values
    '''
    keys = dic.keys()
    values_cp = itertools.product(*dic.values())
    return list(dict(zip(keys, values)) for values in values_cp)

    # df = pd.DataFrame({'a': [[3,4,6]], 'b': [[1]]})
    # for i in df.columns:
    #     df = df.explode(i)

In [ ]:
class LandscapeDataFrame:
    models = ['schwinger']
    reprs = {'schwinger' : ['standard', 'dimensionless']}
    
    parameters = {'schwinger': 
                  {'_general_': ['N', 'H_E', 'H_M', 'H_I', 'theta', 'mixer_type', 'p', 'hash', 'quality'],
                   'standard': ['g', 'm', 'a'],
                   'dimensionless': ['m/g', 'x']
                  }
                 }
    
    parameter_types = {'schwinger':
                       {'N': int, 'theta': float, 'p':int, 'mixer_type': str, 'g': float, 'm': float, 'a': float, 'm/g': float, 'x': float}
                      }
    
    coeff_funcs = {'schwinger': 
                  {'standard': lambda params: {'H_E': (params['g']**2)*params['a']/2,'H_M': params['m']/2,'H_I': 1/(4*params['a'])},
                   'dimensionless': lambda params: {'H_E': 1,'H_M': np.sqrt(params['x'])/2*params['m/g'],'H_I': params['x']/2}
                  }
                 }
    
    def __init__(self, model, rep):
        assert model in self.models, "Not a specified Model"
        assert rep in self.reprs[model], f"Not a represenetation for the {model} model"
        self.model = model.lower()
        self.rep = rep.lower()
        
    def render_landscapes(self, param_ranges, quality):
        param_table = dict_cartesian_product(param_ranges)
        for param in param_table:
            self.render_landscape(param, quality)
    
    def render_landscape(self, params, quality):
        print(*[f"{key}: {str(params[key])}," for key in params])
        
        for key in params:
            type_ = self.parameter_types[self.model][key]
            value = type_(params[key])
            params[key] = value
        
        coeffs = self.coeff_funcs[self.model][self.rep](params)
        params.update(coeffs)
        
        if self.model == 'schwinger':
            hash_params = (params['N'], params['H_E'], params['H_M'], params['H_I'], params['theta'], params['mixer_type'], params['p'])
            
        params['hash'] = hash_iterable(hash_params)
        params['quality']  = quality
        self.load_landscape(params)

    def init_dir(self):
        if not os.path.exists("landscape_data"): # check if main directory exists
            os.makedirs("landscape_data")
        if not os.path.exists(f"landscape_data/{self.model}"): # check if model directory exists
            os.makedirs(f"landscape_data/{self.model}")
        if not os.path.exists(f"landscape_data/{self.model}/data"): # check if model data directory exists
            os.makedirs(f"landscape_data/{self.model}/data")
        for rep in self.parameters[self.model]:
            if not os.path.exists(f"landscape_data/{self.model}/{rep}.csv"): # check if each repr directory for model exists
                fieldnames = self.parameters[self.model][rep]
                if rep != '_general_':
                    fieldnames += self.parameters[self.model]['_general_']
                df = pd.DataFrame(columns=fieldnames)
                df.to_csv(f"landscape_data/{self.model}/{rep}.csv", index=False)
                        
    def save_file(self, landscape, params):
        f = open(f"landscape_data/{self.model}/data/{params['hash']}.npz", 'wb')
        np.savez(f, **landscape)
        f.close()
        
        self.update_csv('_general_', params)
        self.update_csv(self.rep, params)
        
    def update_csv(self, rep, params):
        df = pd.read_csv(f"landscape_data/{self.model}/{rep}.csv") # representation data frame
        row = df[df['hash'] == params['hash']] # data frame row with params

        if row.shape[0] > 0:
            df.at[row.index[0],'quality'] = params['quality']
        else:
            df = df.append(params, ignore_index=True)
            
        df.to_csv(f"landscape_data/{self.model}/{rep}.csv", index=False)
    
    def open_file(self,filename):
        f = open(f"landscape_data/{self.model}/data/{filename}.npz", 'rb')
        npzfile = np.load(f)
        landscape = {'gamma': npzfile['gamma'],'beta': npzfile['beta'],'energy': npzfile['energy']}
        f.close()
        return landscape
        
    def load_landscape(self,params):
        self.init_dir()
        if self.model == 'schwinger':
            target, mixer = self.make_schwinger(params)
        
        df = pd.read_csv(f"landscape_data/{self.model}/_general_.csv") # _general_ data frame
        row = df[df['hash'] == params['hash']] #_general_ data frame row with params
        
        # check if specific schwinger model parameters already exists
        if row.shape[0] == 0: # there is not a row with these parameters
            landscape = self.prime_calc_landscape(target, mixer, params) 
            self.save_file(landscape, params) 
        elif any(params['quality'] > row['quality']):
            landscape = self.prime_calc_landscape(target, mixer, params)
            self.save_file(landscape, params)
        else: # there is a row with these parameters
            print(f"Already Rendered with Quality {row['quality'].iloc[0]}")
            
    def calc_landscape(self, x_range, y_range, energy_f):
        x_quality = len(x_range)
        y_quality = len(y_range)

        x_grid, y_grid = np.meshgrid(x_range, y_range)

        energy = np.zeros([y_quality, x_quality], dtype=float)

        ttl = x_quality * y_quality

        start = time.time()
        for i in range(y_quality):
            for j in range(x_quality):
                energy[i,j] = energy_f(y_grid[i,j], x_grid[i,j])
                proportion = ((i * x_quality) + j + 1) / ttl * 100
                print(f'{proportion:3.2f}% | time remaining: {(time.time() - start) * (100-proportion)/proportion :5.1f}s    ', end='\r', flush = True)

        return energy

    def prime_calc_landscape(self, target, mixer, params):
        assert params.get('quality') is not None, "Require resolution 'quality'"
        assert params.get('p') is not None, "Require number of rounds 'p'"
        
        p = params['p']
        complete_ham = target + mixer
        gamma_range, beta_range = self.landscape_domain(target, mixer, params)

        ansatz = QAOAAnsatz(cost_operator = target, 
                            mixer_operator = mixer,
                            reps = p,
                            name = 'QAOA')
        vqe = VQE(ansatz, quantum_instance=QuantumInstance(backend=Aer.get_backend('statevector_simulator')))
        
        energy_eval = vqe.get_energy_evaluation(complete_ham)
        calc_vqe_energy = lambda gamma, beta: energy_eval([gamma,beta])

        start = time.time()
        energy = self.calc_landscape(gamma_range, beta_range, calc_vqe_energy)
        print(f"Rendered In: {time.time() - start : .2f}s" + '\t' * 20)
        return {'gamma': gamma_range, 'beta': beta_range, 'energy': energy}
    
    def landscape_domain(self, target, mixer, params):
        quality = 2 ** params['quality']
        
        min_gamma_coeff = find_min_coeff(target)
        min_beta_coeff = find_min_coeff(mixer)
        
        gamma_range = np.linspace(0, (2/min_gamma_coeff)*np.pi, int(quality * 1/(min_gamma_coeff)) + 1)
        beta_range = np.linspace(0, (2/min_beta_coeff)*np.pi, int(quality * 1/(min_beta_coeff)) + 1)
        return gamma_range, beta_range
        
    def make_schwinger(self,params):
        target, mixer = make_schwinger(N = params['N'], 
                                       H_E_coeff = params['H_E'],
                                       H_M_coeff = params['H_M'],
                                       H_I_coeff = params['H_I'],
                                       theta = params['theta'], mixer_type = params['mixer_type'])
        return target, mixer